In [45]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [47]:
data.sample(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
20023,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19496,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31954,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24014,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38609,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35230,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39597,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37597,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22610,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33526,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [49]:
data.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [50]:
data.shape

(42000, 785)

In [51]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

In [55]:
# Initialize parameters with He initialization
def init_params():
    W1 = np.random.randn(10, 784) * np.sqrt(2 / 784)
    b1 = np.random.randn(10, 1) - 0.5
    W2 = np.random.randn(10, 10) * np.sqrt(2 / 10)
    b2 = np.random.randn(10, 1) - 0.5
    return W1, b1, W2, b2

# Activation functions
def ReLU(Z):
    return np.maximum(0, Z)

# Softmax with numerical stability
def softmax(Z):
    Z -= np.max(Z, axis=0, keepdims=True)
    exp_Z = np.exp(Z)
    return exp_Z / np.sum(exp_Z, axis=0)

# Derivative of ReLU for backpropagation
def deriv_ReLU(Z):
    return Z > 0

# Forward propagation
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# One-hot encoding the labels with class count
def one_hot(Y, num_classes):
    one_hot_Y = np.zeros((num_classes, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

# Backpropagation
def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y, num_classes=10)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

# Update parameters
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

In [56]:

# Prediction and accuracy functions
def get_prediction(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

# Gradient descent
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 10 == 0:
            print("Iteration:", i)
            print("Accuracy:", get_accuracy(get_prediction(A2), Y))
    return W1, b1, W2, b2

In [57]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 100, 0.1)

Iteration: 0
Accuracy: 0.09492682926829268
Iteration: 10
Accuracy: 0.09646341463414634
Iteration: 20
Accuracy: 0.09646341463414634
Iteration: 30
Accuracy: 0.09646341463414634
Iteration: 40
Accuracy: 0.09646341463414634
Iteration: 50
Accuracy: 0.09646341463414634
Iteration: 60
Accuracy: 0.09646341463414634
Iteration: 70
Accuracy: 0.1037560975609756
Iteration: 80
Accuracy: 0.1037560975609756
Iteration: 90
Accuracy: 0.1037560975609756
